In [2]:
# 네이버금융에서 특정 종목을 검색해보자
# https://finance.naver.com/item/main.nhn?code=068270 <-- 과거 버전
# https://finance.naver.com/item/main.naver?code=068760
# 068270이 셀트리온의 종목코드다
# 다른종목을 보려면 이 코드만 바꾸면 되는 것.

# 셀트리온의 일별 시세 데이터 전부가져오자
# 페이지당 10개씩 표시되는 상황이다
# 첫 페이지는 최신 날짜의 주가이다
# 화면에서 맨뒤를 클릭하면 가장 오래된 날짜의 종가가 표기된다

# 일별 시세 페이지의 맨뒤버튼에 우클릭을 통해 "페이지 소스 보기"하면
# 아래의 주소가 표시된다
# https://finance.naver.com/item/sise_day.naver?code=068760&page=399
# 여기서 페이지와 종목코드를 넣어서 특정 위치로 갈 수 있다


# beautiful soup와 유명 parser 중 하나인 lxml로 크롤링해보자
# pip install beautifulsoup4
# pip install lxml

# bs4에서 가장 중요한 함수는 find_all() , find() 함수일 것이다

# find_all(['검색할 태그'][, class_='클래스 속성값'][, id='아이디 속성값'][, limit=찾을 개수])
# find(['검색할 태그'][, class_='클래스 속성값'][, id='아이디 속성값'])

# find_all 함수는 문서 전체를 대상으로 조건에 맞는 모든 태그를 찾는다
# 아무것도 못 찾을 때엔 find_all은 비어있는 리스트를 반환, find는 None을 반환

In [3]:
# 네이버 금융이 현재 크롤링을 막아놨는데, 접속자의 http 헤더를 없애버리는 식 인듯 함
# https://ekyoo.tistory.com/15 여기서 해결법을 찾았고
# 여기서 헤더에 넣을 데이터를 확인할 수 있음 (User Agent 확인)
# http://m.avalon.co.kr/check.html

In [4]:
# bs4를 이용해 네이버금융의  셀트리온의 맨 뒤 페이지 숫자를 구해보자
# 결과를 아래 실제 소스와 비교하면서 보자
# view-source:https://finance.naver.com/item/sise_day.naver?code=068760&page=1

""" 네이버에서 이 코드 막아서 아래것으로 전환 함
url = 'https://finance.naver.com/item/sise_day.naver?code=068760&page=1'
with urlopen(url) as doc:
    html = BeautifulSoup(doc, 'lxml')
    pgrr = html.find('td', class_='pgRR')
    #print(pgrr.a['href'])
"""

# 네이버 금융의 맨 뒤 페이지 숫자를 구해보자

# bs4의 생성자 첫번째 인수는 HTML,XML파일 경로나 URL
# 두번째 인수는 파싱 방식을 넘겨준다

# find 함수를 통해 class속성이 pgRR인 td태그를 찾으면 pgrr 변수에 반환된다
# 여기 굳이 class_ 라고 되어있는 부분의 파이썬의 기본 class라는 문법과의 중복을 피하기 위함

# pgrr.a['href']는 td태그 하부 a태그의 href 속성값의 문자열을 얻게 되는 것

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import request

headers = ('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36')
url = 'https://finance.naver.com/item/sise_day.naver?code=068760&page=1'
opener = request.build_opener()
opener.addheaders = [headers]
response = opener.open(url)

html = BeautifulSoup(response, 'lxml')
pgrr = html.find('td', class_='pgRR')
print(pgrr.a['href'])

/item/sise_day.naver?code=068760&page=400


In [8]:
# pgrr 변수에 검색 결과가 담겨져 있으니, 
# pgrr의 전체 텍스트를 확인하려면 getText속성을 이용하면 된다

# 다음과같이 prettify() 함수를 호출하면 getText속성값을 계층적으로 출력해준다

print(pgrr.prettify())

<td class="pgRR">
 <a href="/item/sise_day.naver?code=068760&amp;page=400">
  맨뒤
  <img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
 </a>
</td>



In [10]:
# 태그를 제외한 텍스트 부분만 구할 때는 다음처럼 text속성을 이용하면 된다

print(pgrr.text)


맨뒤
				




In [12]:
# pgrr 내용 : pgrr = html.find('td', class_='pgRR')

# 문자열을 리스트로 얻어서 전체 페이지 수를 얻는 걸 단계적으로 진행해보자

s = str(pgrr.a['href']).split('=')
print(s)

['/item/sise_day.naver?code', '068760&page', '400']


In [14]:
# 위에서 '='를 기준으로 split()으로 분리하여 3개 문자열을 리스트로 얻었다
# 리스트의 제일 마지막 원소가 바로 구하려는 전체 페이지 수다

last_page = s[-1]

print(last_page)

400


In [56]:
# 일별 시세의 전체 페이지 수를 구했으므로 첫 페이지부터 마지막까지 모든 걸 읽어보자

# 일별시세는 테이블 형태의 데이터이니, pandas의 read_html()함수를 사용하는게 효과적.

# 한페이지씩 데이터를 읽어 DataFrame에 추가하면 전체 페이지의 일별 시세 데이터를 구할 수 있다

''' 이 코드로 할랬는데, 위에서 발생했던 헤더관련 문제로 작동하지 않아서 아래와 같이 수정함
for page in range(399, int(last_page)+1):
    page_url = '{}&page={}'.format(sise_url, page)
    df = df.append(pd.read_html(page_url)[0])

df = df.dropna()
'''
import pandas as pd
import requests

df = pd.DataFrame()
sise_url = 'https://finance.naver.com/item/sise_day.naver?code=068760'

for page in range(399, int(last_page)+1):
    page_url = '{}&page={}'.format(sise_url, page)
    df = df.append(pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0])

df = df.dropna() # 값이 없는 행을 제거한다

df


C:\Users\mrkkokku\AppData\Local\Temp/ipykernel_33632/3219413235.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0])
C:\Users\mrkkokku\AppData\Local\Temp/ipykernel_33632/3219413235.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)[0])


,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.03.30,99100.0,500.0,99500.0,100500.0,98400.0,107065.0
2,2022.03.29,98600.0,1400.0,97500.0,99300.0,97300.0,91445.0
3,2022.03.28,97200.0,1800.0,98700.0,98900.0,97100.0,102274.0
4,2022.03.25,99000.0,900.0,98700.0,100800.0,97600.0,165260.0
5,2022.03.24,98100.0,800.0,98400.0,99500.0,96600.0,112585.0
9,2022.03.23,98900.0,400.0,99700.0,100300.0,98400.0,130947.0
10,2022.03.22,98500.0,3500.0,97400.0,100200.0,97300.0,251835.0
11,2022.03.21,102000.0,3000.0,105600.0,105600.0,102000.0,165067.0
12,2022.03.18,105000.0,700.0,106400.0,107600.0,103700.0,288020.0
13,2022.03.17,104300.0,6800.0,99900.0,106000.0,99600.0,398516.0
